In [7]:
import pandas as pd
import json 


df = pd.read_csv('final.csv')

df.head()

,PredictionString,image_id
0,8 0.9396691918373108 172.986572265625 235.0530...,batch_01_vt/0021.jpg
1,8 0.9135417342185974 158.7433319091797 57.3150...,batch_01_vt/0028.jpg
2,1 0.8247161507606506 71.08919525146484 169.160...,batch_01_vt/0031.jpg
3,8 0.8648725152015686 32.01179885864258 125.683...,batch_01_vt/0032.jpg
4,8 0.940662682056427 149.37664794921875 0.19925...,batch_01_vt/0070.jpg


In [8]:
print(len(df['image_id']))

837


In [9]:
import numpy 

pred= df['PredictionString'][0]


test= pred.split()
test = map(float, test)
test = list(test)
test = numpy.array(test).reshape(6,-1)
for d in test:
    if d[1] > 0.75:
        print(d)
#print(test[0, 1])

[8.00000000e+00 9.39669192e-01 1.72986572e+02 2.35053085e+02
 4.61859467e+02 4.70989594e+02 2.00000000e+00 1.54796854e-01
 1.51095032e+02 6.31322746e+01]
[2.55048859e+02 1.65999908e+02 1.00000000e+00 7.37537146e-02
 1.51673416e+02 6.31665154e+01 2.55493103e+02 1.65492981e+02
 6.00000000e+00 4.77585718e-02]
[3.94999817e+02 3.65999878e+02 4.40999878e+02 4.30999786e+02
 8.00000000e+00 3.65048572e-02 2.41999893e+02 2.56000244e+02
 4.00000031e+02 4.42000153e+02]
[4.43000092e+02 4.31999725e+02 1.00000000e+00 2.52760015e-02
 1.84798874e+02 3.63593475e+02 2.08691330e+02 3.98619141e+02
 1.00000000e+00 1.73583795e-02]
[2.27533800e+02 1.54430050e+02 2.51068950e+02 2.40088670e+02
 1.00000000e+00 1.38099748e-02 5.17362000e-01 3.33989350e+02
 8.11788000e+01 4.49212650e+02]


In [10]:
from collections import OrderedDict

data = OrderedDict()


pseudo_images=[]
size = len(df['image_id'])

for i in range(size):
    pseudo_images.append(
        {
            "license": "",
            "url": "",
            "file_name": df['image_id'][i],
            "height": 512,
            "width": 512,
            "date_captured": 0,
            "id": i
            
        }
    )
data["images"] = pseudo_images

In [11]:
pseudo_annotations =[]
pred= df['PredictionString'][11]


bbox = pred.split()
bbox = map(float, bbox)
bbox = list(bbox)
print(bbox)
bbox = numpy.array(bbox).reshape(-1,6)
print(bbox.shape)
for d in bbox:
    if d[1] > 0.75:        
        x, y, w, h = tuple(d[2:])
        id = int(d[0])
        pseudo_annotations.append(
            {
                "segmentation": "",
                "area": w * h,
                "image_id": i,
                "category_id": id,
                "bbox": [x, y, w, h],
                "iscrowd": 0
            }
        )
#print(pseudo_annotations)
data["annotations"] = pseudo_annotations

[8.0, 0.8713299632072449, 157.2597198486328, 230.3306884765625, 378.59722900390625, 393.18499755859375, 6.0, 0.06721837818622589, 157.3400115966797, 231.5853729248047, 378.0379638671875, 394.86767578125, 8.0, 0.015433830952380953, 100.00061, 117.00389, 420.63608, 412.1353]
(3, 6)


In [154]:
df['PredictionString'][11]

'8 0.8713299632072449 157.2597198486328 230.3306884765625 378.59722900390625 393.18499755859375 6 0.06721837818622589 157.3400115966797 231.5853729248047 378.0379638671875 394.86767578125 8 0.015433830952380953 100.00061 117.00389 420.63608 412.1353 '

In [15]:
from collections import OrderedDict
import numpy

data = OrderedDict()

pseudo_annotations =[]
pseudo_images=[]
size = len(df['image_id'])

#size =11
cnt = 0
for i in range(size):
    pseudo_images.append(
        {
            "license": "",
            "url": "",
            "file_name": df['image_id'][i],
            "height": 512,
            "width": 512,
            "date_captured": 0,
            "id": i
            
        }
    )
    pred= df['PredictionString'][i]
    bbox = pred.split()
    bbox = map(float, bbox)
    bbox = list(bbox)
    bbox = numpy.array(bbox).reshape(-1,6)
   
 #   cnt =0
    for d in bbox:
        if d[1] > 0.75:
            x_min, y_min, x_max, y_max = tuple(d[2:6])
            w = x_max-x_min
            h  = y_max - y_min
            #id = int(d[0])
            pseudo_annotations.append(
                {
                    "id": cnt,
                    "segmentation": 0,
                    "area": w * h,
                    "image_id": i,
                    "category_id": id,
                    "bbox": [x_min, y_min, w, h],
                    "iscrowd": 0
                }
            )

            cnt +=1
    
data["images"] = pseudo_images
data["annotations"] = pseudo_annotations


In [16]:
cls = [
    'UNKNOWN',
    'General trash',
    'Paper',
    'Paper pack',
    'Metal',
    'Glass',
    'Plastic',
    'Styrofoam',
    'Plastic bag',
    'Battery',
    'Clothing'
]

pseudo_categories =[]
for i in range(len(cls)):

    pseudo_categories.append(
        {
            "id": i,
            "name": cls[i],
            "supercategory": cls[i]
        }
    )

data["categories"] = pseudo_categories


In [17]:
with open('pseudo5.json', 'w', encoding="utf-8") as make_file:
    json.dump(data, make_file, ensure_ascii=False, indent="\t")